In [47]:
# Imports
import numpy as np
from pydrake.all import *
from IPython.display import display, SVG, clear_output, HTML

# Import pygmo for optimisation
import pygmo as pg
from urdf_creator import *
import time

from numba import jit, float64
import pickle

from scipy.optimize import differential_evolution, rosen

In [50]:
bounds = np.array([(0,2), (0, 2), (0, 2), (0, 2), (0, 2)])

result = differential_evolution(rosen, bounds)

print(result.x, result.fun)

[1. 1. 1. 1. 1.] 0.0


In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [17]:
# Define the number of actuators
na = 7

# GLOBAL PROBLEM SETUP

class ActuatorBox(LeafSystem):
    # Input values are for motor and NOT for the joint
    def __init__(self, tau_max_m, omega_max_m, gear_ratio, k_velocity):
        LeafSystem.__init__(self)
        # Both the values are motor specific and NOT joint specific
        # Get joint max torque
        self.tau_max_j = tau_max_m*gear_ratio
        self.k_vel = k_velocity
        # Convert RPM of motor to joint velocity in rad/sec
        self.omega_max_j = omega_max_m*(2*np.pi/60)/gear_ratio
        self.DeclareVectorInputPort('commanded_torque', BasicVector(na))
        self.DeclareVectorInputPort('joint_state', BasicVector(2*na))
        self.DeclareVectorOutputPort('realised_torque', BasicVector(na), self.OutputJointTorque)

    def OutputJointTorque(self, context, output):
        # Get the state values
        q = self.EvalVectorInput(context, 1).get_value()
        qvel = q[na:]
        u = self.EvalVectorInput(context, 0).get_value()
        # Compute the instantaneous torque limit
        # For the AK-70-10 motors
        kv = self.k_vel
        u_new = np.copy(u)
        # Using URDF safety controller rules
        for i in range(len(u)):
            if qvel[i]>=0 and u[i]>=0:
                u_bound = -kv*(qvel[i]-self.omega_max_j)
                u_new[i] = np.min([u[i], u_bound, self.tau_max_j])
            if qvel[i]>=0 and u[i]<=0:
                u_bound = -self.tau_max_j
                u_new[i] = np.max([u[i], u_bound])
            if qvel[i]<=0 and u[i]>=0:
                u_bound = self.tau_max_j
                u_new[i] = np.min([u[i], u_bound])
            if qvel[i]<=0 and u[i]<=0:
                u_bound = -(-kv*(np.abs(qvel[i])-self.omega_max_j))
                u_new[i] = np.max([u[i], u_bound, -self.tau_max_j])
#             print(u[i], u_bound, u_new[i])
        output.SetFromVector(u_new)

# Task definition
# Home position
q0 = np.array([-0.64, -0.35, -1.01,  1.7, -0.29 ,0.88, -0.034])
carti = np.array([-0.53, 0.13, 0.81])
cartd = np.array([-0.23, 0.24, 1.03])

# Simulation settings
time_step = 1e-4
sim_time = 1
error_threshold = 5e-3

# Robot URDF
urdf_path = './urdfs/divar3v7308465156.urdf'
print(urdf_path)

# motor mass
m_m = 0.452
# Assumed values
tau_mu = 5.3e-3
b0 = 1.5e-5
# AK70-10 parameters
# gear ratio
gear_ratio = 10
# motor design variables
tau_max_m = 12/gear_ratio # Nm
omega_max_m = 475*gear_ratio   # RPM
Kt = 0.095        # Nm/A
Kv = 100          # RPM/V
Km = 0.19         # Nm/sqrt(W)
# Computed matrix for joule loss
K_joule = 1/np.sqrt(Km)*np.identity(na)
k_velocity = 8.33/(75*2*np.pi/60) # Nm/(rad/s)
# Control attributes
Kp = 15
Kd = 2*np.sqrt(Kp)
max_penal = 1e4


# For baseline-codesign study
pre_val_names = ["tau_mu", "b0", "k_velocity"]
pre_val_vals = [0.0053, 1.5e-05, 1.0606085407643906]
# Define a dictionary for ease of usage and modulartiy of the dvs
pre_val = dict(zip(pre_val_names, pre_val_vals))
dv_names = ["omega_max_m", "gear_ratio", "Kp", "Kd", "m_m", "a1", "lua","a23_distr", "lfa", "a45_distr", "a7"]
print('pre_val: ', pre_val, 'dv_names: ', dv_names)
sample_dv = [4750, 10, 15, 2*np.sqrt(15), 0.452, 0.36, 0.36,0.5,0.36,0.5,0.1]
print('Sample-DV: ', sample_dv)
dv_bounds = ([1000,0.1, 0, 0, 1e-3,1e-2,0.1,0.1,0.1,0.1,0.1],[9000,25,100,100,2,0.5,0.5,0.9,0.5,0.9,0.25])
print('dv_bounds: ', dv_bounds)  

./urdfs/divar3v7308465156.urdf
pre_val:  {'tau_mu': 0.0053, 'b0': 1.5e-05, 'k_velocity': 1.0606085407643906} dv_names:  ['omega_max_m', 'gear_ratio', 'Kp', 'Kd', 'm_m', 'a1', 'lua', 'a23_distr', 'lfa', 'a45_distr', 'a7']
Sample-DV:  [4750, 10, 15, 7.745966692414834, 0.452, 0.36, 0.36, 0.5, 0.36, 0.5, 0.1]
dv_bounds:  ([1000, 0.1, 0, 0, 0.001, 0.01, 0.1, 0.1, 0.1, 0.1, 0.1], [9000, 25, 100, 100, 2, 0.5, 0.5, 0.9, 0.5, 0.9, 0.25])


In [36]:
a = [(1,1),(2,2)]

In [40]:
type(a[0])

tuple

In [42]:
type((np.array(dv_bounds).T)[0])

numpy.ndarray

In [18]:
pre_val_vals = [tau_mu, b0, Km, k_velocity, tau_max_m, omega_max_m, gear_ratio, m_m]
pre_val_names = ["tau_mu", "b0", "Km", "k_velocity", "tau_max_m", "omega_max_m", "gear_ratio", "m_m"]
# Define a dictionary for ease of usage and modulartiy of the dvs
pre_val = dict(zip(pre_val_names, pre_val_vals))

task_info = {'carti': carti, 'cartd': cartd}
pre_val.update(task_info)

In [25]:
def compute_QoI(pre_val, dv_names, sample_dv, run_sim=False):
    
    class ActuatorBox(LeafSystem):
        # Input values are for motor and NOT for the joint
        def __init__(self, tau_max_m, omega_max_m, gear_ratio, k_velocity):
            LeafSystem.__init__(self)
            # Both the values are motor specific and NOT joint specific
            # Get joint max torque
            self.tau_max_j = tau_max_m*gear_ratio
            self.k_vel = k_velocity
            # Convert RPM of motor to joint velocity in rad/sec
            self.omega_max_j = omega_max_m*(2*np.pi/60)/gear_ratio
            self.DeclareVectorInputPort('commanded_torque', BasicVector(na))
            self.DeclareVectorInputPort('joint_state', BasicVector(2*na))
            self.DeclareVectorOutputPort('realised_torque', BasicVector(na), self.OutputJointTorque)

        def OutputJointTorque(self, context, output):
            # Get the state values
            q = self.EvalVectorInput(context, 1).get_value()
            qvel = q[na:]
            u = self.EvalVectorInput(context, 0).get_value()
            # Compute the instantaneous torque limit
            # For the AK-70-10 motors
            kv = self.k_vel
            u_new = np.copy(u)
            # Using URDF safety controller rules
            for i in range(len(u)):
                if qvel[i]>=0 and u[i]>=0:
                    u_bound = -kv*(qvel[i]-self.omega_max_j)
                    u_new[i] = np.min([u[i], u_bound, self.tau_max_j])
                if qvel[i]>=0 and u[i]<=0:
                    u_bound = -self.tau_max_j
                    u_new[i] = np.max([u[i], u_bound])
                if qvel[i]<=0 and u[i]>=0:
                    u_bound = self.tau_max_j
                    u_new[i] = np.min([u[i], u_bound])
                if qvel[i]<=0 and u[i]<=0:
                    u_bound = -(-kv*(np.abs(qvel[i])-self.omega_max_j))
                    u_new[i] = np.max([u[i], u_bound, -self.tau_max_j])
    #             print(u[i], u_bound, u_new[i])
            output.SetFromVector(u_new)    
    
    # Construct the master dv list
    dv_dict = dict(zip(dv_names, sample_dv))
    # Map the entire design space from the sampled DVs using heuristics
    # Current heuristics used are from the jump paper
    dv_dict['tau_max_m'] = 5.48*dv_dict['m_m']**0.97
    dv_dict['Km'] = 0.15*dv_dict['m_m']**1.39
    # Later also fit k_velocity values
    dv = {**pre_val, **dv_dict}
    urdf_path, updated_dv =urdf_creator(dv)
    dv = updated_dv
    # Construct the sim diagram
    builder = DiagramBuilder()
    scene_graph = builder.AddSystem(SceneGraph())
    plant = builder.AddSystem(MultibodyPlant(time_step=time_step))
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, 'diva_robot')
    X_R = RigidTransform(RotationMatrix.MakeYRotation(-np.pi/2), np.array([-0.1, 0.5, 1.2]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)

    plant.Finalize()
#     actuator = builder.AddSystem(ActuatorBox(dv['tau_max_m'], dv['omega_max_m'], dv['gear_ratio'], dv['k_velocity']))
    actuator = builder.AddSystem(ActuatorBox(dv['tau_max_m'], dv['omega_max_m'], dv['gear_ratio'], dv['k_velocity']))
    
    # Initialise controller params, for now limited to Kp and Kd search
    Kp = np.full(na, dv['Kp'])
    Ki = np.full(na, 0)
    Kd = np.full(na, dv['Kd'])
    
    iiwa_controller = builder.AddSystem(InverseDynamicsController(plant, Kp, Ki, Kd, False))
    iiwa_controller.set_name("iiwa_controller");
    # Complete connections
    builder.Connect(plant.get_state_output_port(model),
                iiwa_controller.get_input_port_estimated_state())
    builder.Connect(iiwa_controller.get_output_port_control(), 
                actuator.get_input_port(0))
    builder.Connect(plant.get_state_output_port(model),
                actuator.get_input_port(1))
    builder.Connect(actuator.get_output_port(), plant.get_actuation_input_port())
    
#     # Connecting a data logger
#     # Commanded torque
#     logger1 = LogVectorOutput(iiwa_controller.get_output_port_control(), builder)
#     # Joint state
#     logger2 = LogVectorOutput(plant.get_state_output_port(model), builder)
#     # Realised torque
#     logger3 = LogVectorOutput(actuator.get_output_port(), builder)
    
    # Instead trying to read the individual port values

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    gripper_frame = plant.GetBodyByName("eef").body_frame()
    
    # Now computing the QoIs
    # WORKSPACE
    # Workspace is always satisfied since its the same robot
    # Compute IK - pos1
    ik = InverseKinematics(plant, plant_context)
    ik.AddPositionConstraint(
                gripper_frame, [0, 0, 0], plant.world_frame(),
                dv['carti'], dv['carti'])
    # Adding the orientation constraint to keep the end-effector
    # perpendicular to the table
    ik.AddOrientationConstraint(gripper_frame, 
                                RotationMatrix(), 
                                plant.world_frame(),
                               RollPitchYaw([-3.14, 0, 0]).ToRotationMatrix(), 0.0)
    
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    # Solve
    solver = SnoptSolver()
    result = solver.Solve(ik.prog())
    qi = result.GetSolution(ik.q())
    qi = (np.arctan2(np.sin(qi), np.cos(qi)))
    plant.SetPositions(plant.GetMyContextFromRoot(context),model,qi)
    eef_pos_realised = plant.GetBodyByName('eef').EvalPoseInWorld(plant_context).translation()
    w1 = np.linalg.norm(eef_pos_realised-dv['carti'])
    
    # Compute IK - pos2
    ik = InverseKinematics(plant, plant_context)
    ik.AddPositionConstraint(
                gripper_frame, [0, 0, 0], plant.world_frame(),
                dv['cartd'], dv['cartd'])
    ik.AddOrientationConstraint(gripper_frame, 
                                RotationMatrix(), 
                                plant.world_frame(),
                               RollPitchYaw([-3.14, 0, 0]).ToRotationMatrix(), 0.0)
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.AddQuadraticErrorCost(np.identity(len(q)), qi, q)
    prog.SetInitialGuess(q, qi)
    solver = SnoptSolver()
    result = solver.Solve(ik.prog())
    qd = result.GetSolution(ik.q())
    qd = (np.arctan2(np.sin(qd), np.cos(qd)))
    plant.SetPositions(plant.GetMyContextFromRoot(context),model,qd)
    eef_pos_realised = plant.GetBodyByName('eef').EvalPoseInWorld(plant_context).translation()
    w2 = np.linalg.norm(eef_pos_realised-dv['cartd'])
    # Compute the error in positions    
    workspace = w1+w2
    # APPLIED FORCE
    applied_force = 0
    # PAYLOAD
    # same as applied force
    payload = 0
    # Setup the simulation
    xd = np.hstack((qd, 0*qd))
    plant.SetPositions(plant_context, q0)
    iiwa_controller.GetInputPort('desired_state').FixValue(
    iiwa_controller.GetMyMutableContextFromRoot(context), xd);
    # Simulation
    simulator = Simulator(diagram, context)
    simulator.AdvanceTo(sim_time);
    
#     # Now we want to get the simulation data, i.e., 
#     # the torque applied, final time of the joint variables
#     log1 = logger1.FindLog(context)
#     log2 = logger2.FindLog(context)
#     log3 = logger3.FindLog(context)
    
#     error = np.abs(log2.data()[:na,:].transpose() - np.tile(qd,(log3.data().shape[1],1)))
#     e_norm = np.array([np.linalg.norm(val) for val in error])
#     for i in range(len(e_norm)):
#         if np.all(e_norm[i:]<error_threshold):
#             break
#     final_error = e_norm[-1]
#     # CYCLE TIME
#     ts = log2.sample_times()[i]
    
#     omega_j = []
#     for i in range(log2.data().shape[1]):
#         omega_j.append(log2.data()[:,i][na:])
#     omega_j = np.array(omega_j).transpose()
#     omega_m = omega_j*dv['gear_ratio']
#     # Convert the motor speed to RPM
#     omega_m_rpm = omega_m*60/(2*np.pi)
#     # Motor torque realised
#     tau_m = log3.data()/dv['gear_ratio']
    
#     K_joule = 1/np.sqrt(dv['Km'])*np.identity(na)
#     # Total power losses
#     inst_friction_power = []
#     inst_joule_power = []
#     for i in range(len(log3.sample_times())):
#         omega_inst = omega_m[:, i]
#         tau_f = dv['tau_mu']*np.sign(omega_inst)+dv['b0']*omega_inst
#         P_f = np.dot(tau_f, omega_inst)
#         inst_friction_power.append(P_f)
#         tau_t = tau_f+tau_m[:, i]
#         P_t = np.dot(K_joule.dot(tau_t), tau_t)
#         inst_joule_power.append(P_t)
    
#     # POWER LOSSES
#     total_friction_loss = np.sum(inst_friction_power)*time_step
#     total_joule_loss = np.sum(inst_joule_power)*time_step
    
#     results = np.array([workspace, 0, 0, 0, ts, total_friction_loss, total_joule_loss, final_error])
    results = np.array([workspace, 0, 0, 0, 0, 0, 0, 0])
    return results

In [26]:
# ActuatorBox = ActuatorBox()
# compute_QoI(pre_val, dv_names, sample_dv, ActuatorBox, run_sim=True)

In [27]:
# Setting up a problem
class Baseline_Problem:
    def __init__(self, pre_val, dv_names, dv_bounds):
        self.pre_val = pre_val
        self.dv_names = dv_names
        self.dv_bounds = dv_bounds
        pass

    def fitness(self,sample_dv):
        # print(sample_dv)
        # Write a function to output the QoI values
        results = compute_QoI(self.pre_val, self.dv_names, sample_dv)
        # Add weighting factor, K_w
        # workspace, feasibility, deflection, payload, ts, total_friction_loss, total_joule_loss, final_error
        K_w = np.diag([1,0,0,0,100,5,5,0])
        # Quadratic objective meta function with weights
        fitness_val = np.matmul(K_w, results).dot(results)
        return [fitness_val]

    def get_bounds(self):
        # Define bounds on the controller variables
        # Ideally from a csv file like before so that one can easily extend the optimisation problem
        return self.dv_bounds

    def get_name(self):
        # get the name of the problem also from the csv sheet
        return "Baseline_Codesign"

In [28]:
import ray
from ray.util.multiprocessing import Pool

# import pathos.pools as pp
# p = pp.ProcessPool(4)


def _evolve_func(algo, pop): # doctest : +SKIP
    new_pop = algo.evolve(pop)
    return algo, new_pop

class cool_mp_island(object): # doctest : +SKIP
    def __init__(self):
        # Init the process pool, if necessary.
#         pg.mp_island.init_pool()
        self.pool = Pool()
#         self.pool = pp.ProcessPool()
        pass

    def run_evolve(self, algo, pop):
#         with mp_island._pool_lock:
        res = self.pool.map(_evolve_func, (algo, pop))
#         res = mp_island._pool.apply_async(_evolve_func, (algo, pop))
        return res.get()

In [29]:
# Using CMA-ES
prob = pg.problem(Baseline_Problem(pre_val, dv_names, dv_bounds))
algo = pg.algorithm(pg.cmaes(gen = 2, sigma0=0.3, force_bounds=True))
algo.set_verbosity(100)

# Writing a custom island to see if we can make it work
# The default mp_island fails because of the pickle error from PyCapsule
# caused by drake AddMultibodyPlantSceneGraph function
class cool_isl:
    def run_evolve(self, algo, pop):
        # Island serially runs to evolve the pop
        new_pop = algo.evolve(pop)
        return algo, new_pop
    def get_name(self):
        return "This is a cool island that works"  

In [30]:
%%time
# archi = pg.archipelago(n=1, algo = algo, prob=prob, pop_size=3, udi=cool_mp_island())
archi = pg.archipelago(n=2, algo = algo, prob=prob, pop_size=1, udi=pg.mp_island())
# archi = pg.archipelago(n=1, algo = algo, prob=prob, pop_size=500, udi=cool_isl())
archi.evolve();
archi.wait_check()

RuntimeError: The asynchronous evolution of a pythonic island of type 'Multiprocessing island' raised an error:
Traceback (most recent call last):
  File "/home/laniakea/anaconda3/envs/diva/lib/python3.8/site-packages/pygmo/_py_islands.py", line 203, in run_evolve
    ser_algo_pop = pickle.dumps((algo, pop))
  File "/home/laniakea/anaconda3/envs/diva/lib/python3.8/site-packages/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/laniakea/anaconda3/envs/diva/lib/python3.8/site-packages/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle 'PyCapsule' object


In [31]:
print(archi)

Number of islands: 2
Topology: Unconnected
Migration type: point-to-point
Migrant handling policy: preserve
Status: idle

Islands summaries:

	#  Type                    Algo                                                        Prob               Size  Status  
	------------------------------------------------------------------------------------------------------------------------
	0  Multiprocessing island  CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy  Baseline_Codesign  1     idle    
	1  Multiprocessing island  CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy  Baseline_Codesign  1     idle    



In [360]:
a = archi.get_champions_f()
a2 = sorted(archi.get_champions_f(), key = lambda x: x[0])[0]
best_isl_idx = [(el == a2).all() for el in a].index(True)
x_best = archi.get_champions_x()[best_isl_idx]
f_best = archi.get_champions_f()[best_isl_idx]

print(dv_names)
print(x_best)

['omega_max_m', 'gear_ratio', 'Kp', 'Kd', 'm_m', 'a1', 'lua', 'a23_distr', 'lfa', 'a45_distr', 'a7']
[8.05701319e+03 1.86784857e+01 6.60986122e+01 6.37933415e+01
 1.28294220e+00 2.34940097e-01 4.21124955e-01 2.34352043e-01
 4.10181682e-01 5.64641268e-01 1.33269698e-01]


In [381]:
opt_sample = np.array([8.05701319e+03, 1.86784857e+01, 6.60986122e+01, 6.37933415e+01,
       1.28294220e+00, 2.34940097e-01, 4.21124955e-01, 2.34352043e-01,
       4.10181682e-01, 5.64641268e-01, 1.33269698e-01])

results = viz_optimal(pre_val, dv_names, opt_sample)
print(results)
K_w = np.diag([1,0,0,0,100,5,5,0])
# Quadratic objective meta function with weights
fitness_val = np.matmul(K_w, results).dot(results)
print(fitness_val)
# ([workspace, 0, 0, 0, ts, total_friction_loss, total_joule_loss, 0])

RuntimeError: OutputPort: The Context given as an argument was not created for this OutputPort[0] (realised_torque) of System ::_::__main__.ActuatorBox@00005600f1cc6790 (__main__.ActuatorBox)

In [380]:
# Define the number of actuators
na = 7

# GLOBAL PROBLEM SETUP

class ActuatorBox(LeafSystem):
    # Input values are for motor and NOT for the joint
    def __init__(self, tau_max_m, omega_max_m, gear_ratio, k_velocity):
        LeafSystem.__init__(self)
        # Both the values are motor specific and NOT joint specific
        # Get joint max torque
        self.tau_max_j = tau_max_m*gear_ratio
        self.k_vel = k_velocity
        # Convert RPM of motor to joint velocity in rad/sec
        self.omega_max_j = omega_max_m*(2*np.pi/60)/gear_ratio
        self.DeclareVectorInputPort('commanded_torque', BasicVector(na))
        self.DeclareVectorInputPort('joint_state', BasicVector(2*na))
        self.DeclareVectorOutputPort('realised_torque', BasicVector(na), self.OutputJointTorque)

    def OutputJointTorque(self, context, output):
        # Get the state values
        q = self.EvalVectorInput(context, 1).get_value()
        qvel = q[na:]
        u = self.EvalVectorInput(context, 0).get_value()
        # Compute the instantaneous torque limit
        # For the AK-70-10 motors
        kv = self.k_vel
        u_new = np.copy(u)
        # Using URDF safety controller rules
        for i in range(len(u)):
            if qvel[i]>=0 and u[i]>=0:
                u_bound = -kv*(qvel[i]-self.omega_max_j)
                u_new[i] = np.min([u[i], u_bound, self.tau_max_j])
            if qvel[i]>=0 and u[i]<=0:
                u_bound = -self.tau_max_j
                u_new[i] = np.max([u[i], u_bound])
            if qvel[i]<=0 and u[i]>=0:
                u_bound = self.tau_max_j
                u_new[i] = np.min([u[i], u_bound])
            if qvel[i]<=0 and u[i]<=0:
                u_bound = -(-kv*(np.abs(qvel[i])-self.omega_max_j))
                u_new[i] = np.max([u[i], u_bound, -self.tau_max_j])
#             print(u[i], u_bound, u_new[i])
        output.SetFromVector(u_new)

# Task definition
# Home position
q0 = np.array([-0.64, -0.35, -1.01,  1.7, -0.29 ,0.88, -0.034])
carti = np.array([-0.53, 0.13, 0.81])
cartd = np.array([-0.23, 0.24, 1.03])

# Simulation settings
time_step = 1e-4
sim_time = 1
error_threshold = 5e-3

# Robot URDF
urdf_path = './urdfs/divar3v7308465156.urdf'
print(urdf_path)

# motor mass
m_m = 0.452
# Assumed values
tau_mu = 5.3e-3
b0 = 1.5e-5
# AK70-10 parameters
# gear ratio
gear_ratio = 10
# motor design variables
tau_max_m = 12/gear_ratio # Nm
omega_max_m = 475*gear_ratio   # RPM
Kt = 0.095        # Nm/A
Kv = 100          # RPM/V
Km = 0.19         # Nm/sqrt(W)
# Computed matrix for joule loss
K_joule = 1/np.sqrt(Km)*np.identity(na)
k_velocity = 8.33/(75*2*np.pi/60) # Nm/(rad/s)
# Control attributes
Kp = 15
Kd = 2*np.sqrt(Kp)
max_penal = 1e4

# For baseline-codesign study
pre_val_names = ["tau_mu", "b0", "k_velocity"]
pre_val_vals = [0.0053, 1.5e-05, 1.0606085407643906]
# Define a dictionary for ease of usage and modulartiy of the dvs
pre_val = dict(zip(pre_val_names, pre_val_vals))
dv_names = ["omega_max_m", "gear_ratio", "Kp", "Kd", "m_m", "a1", "lua","a23_distr", "lfa", "a45_distr", "a7"]
print('pre_val: ', pre_val, 'dv_names: ', dv_names)
sample_dv = [4750, 10, 15, 2*np.sqrt(15), 0.452, 0.36, 0.36,0.5,0.36,0.5,0.1]
print('Sample-DV: ', sample_dv)
dv_bounds = ([1000,0.1, 0, 0, 1e-3,1e-2,0.1,0.1,0.1,0.1,0.1],[9000,25,100,100,2,0.5,0.5,0.9,0.5,0.9,0.25])
print('dv_bounds: ', dv_bounds)  

task_info = {'carti': carti, 'cartd': cartd}
pre_val.update(task_info)

def viz_optimal(pre_val, dv_names, sample_dv):
    # Construct the master dv list
    dv_dict = dict(zip(dv_names, sample_dv))
    # Map the entire design space from the sampled DVs using heuristics
    # Current heuristics used are from the jump paper
    dv_dict['tau_max_m'] = 5.48*dv_dict['m_m']**0.97
    dv_dict['Km'] = 0.15*dv_dict['m_m']**1.39
    # Later also fit k_velocity values
    dv = {**pre_val, **dv_dict}
    urdf_path, updated_dv =urdf_creator(dv)
    dv = updated_dv
    # Construct the sim diagram
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
    
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, 'diva_robot')
    X_R = RigidTransform(RotationMatrix.MakeYRotation(-np.pi/2), np.array([-0.1, 0.5, 1.2]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)

    meshcat.Delete()
    meshcat.DeleteAddedControls()
    # Spawn table
    table = Parser(plant, scene_graph).AddModelFromFile('./urdfs/table/extra_heavy_duty_table_modified.sdf','table')

    plant.Finalize()
    actuator = builder.AddSystem(ActuatorBox(dv['tau_max_m'], dv['omega_max_m'], dv['gear_ratio'], dv['k_velocity']))
    
    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
    
    # Initialise controller params, for now limited to Kp and Kd search
    Kp = np.full(na, dv['Kp'])
    Ki = np.full(na, 0)
    Kd = np.full(na, dv['Kd'])
    
    iiwa_controller = builder.AddSystem(InverseDynamicsController(plant, Kp, Ki, Kd, False))
    iiwa_controller.set_name("iiwa_controller");
    # Complete connections
    builder.Connect(plant.get_state_output_port(model),
                iiwa_controller.get_input_port_estimated_state())
    builder.Connect(iiwa_controller.get_output_port_control(), 
                actuator.get_input_port(0))
    builder.Connect(plant.get_state_output_port(model),
                actuator.get_input_port(1))
    builder.Connect(actuator.get_output_port(), plant.get_actuation_input_port())
    
    # Connecting a data logger
    # Commanded torque
    logger1 = LogVectorOutput(iiwa_controller.get_output_port_control(), builder)
    # Joint state
    logger2 = LogVectorOutput(plant.get_state_output_port(model), builder)
    # Realised torque
    logger3 = LogVectorOutput(actuator.get_output_port(), builder)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    gripper_frame = plant.GetBodyByName("eef").body_frame()
    print(actuator.get_output_port().EvalBasicVector(context))
    
    # Now computing the QoIs
    # WORKSPACE
    # Workspace is always satisfied since its the same robot
    # Compute IK - pos1
    ik = InverseKinematics(plant, plant_context)
    ik.AddPositionConstraint(
                gripper_frame, [0, 0, 0], plant.world_frame(),
                dv['carti'], dv['carti'])
    # Adding the orientation constraint to keep the end-effector
    # perpendicular to the table
    ik.AddOrientationConstraint(gripper_frame, 
                                RotationMatrix(), 
                                plant.world_frame(),
                               RollPitchYaw([-3.14, 0, 0]).ToRotationMatrix(), 0.0)
    
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    qi = result.GetSolution(ik.q())
    qi = (np.arctan2(np.sin(qi), np.cos(qi)))
    plant.SetPositions(plant.GetMyContextFromRoot(context),model,qi)
    eef_pos_realised = plant.GetBodyByName('eef').EvalPoseInWorld(plant_context).translation()
    w1 = np.linalg.norm(eef_pos_realised-dv['carti'])
    
    # Compute IK - pos2
    ik = InverseKinematics(plant, plant_context)
    ik.AddPositionConstraint(
                gripper_frame, [0, 0, 0], plant.world_frame(),
                dv['cartd'], dv['cartd'])
    ik.AddOrientationConstraint(gripper_frame, 
                                RotationMatrix(), 
                                plant.world_frame(),
                               RollPitchYaw([-3.14, 0, 0]).ToRotationMatrix(), 0.0)
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.AddQuadraticErrorCost(np.identity(len(q)), qi, q)
    prog.SetInitialGuess(q, qi)
    result = Solve(ik.prog())
    qd = result.GetSolution(ik.q())
    qd = (np.arctan2(np.sin(qd), np.cos(qd)))
    plant.SetPositions(plant.GetMyContextFromRoot(context),model,qd)
    eef_pos_realised = plant.GetBodyByName('eef').EvalPoseInWorld(plant_context).translation()
    w2 = np.linalg.norm(eef_pos_realised-dv['cartd'])
    # Compute the error in positions    
    workspace = w1+w2
    # APPLIED FORCE
    # Applied force will also remain the same given that the tau_max is fixed
    applied_force = 0
    # PAYLOAD
    # same as applied force
    payload = 0
    # Setup the simulation
    xd = np.hstack((qd, 0*qd))
    plant.SetPositions(plant_context, qi)
    iiwa_controller.GetInputPort('desired_state').FixValue(
    iiwa_controller.GetMyMutableContextFromRoot(context), xd);
    # Simulation
    simulator = Simulator(diagram, context)
    simulator.AdvanceTo(sim_time);
    
    # Now we want to get the simulation data, i.e., 
    # the torque applied, final time of the joint variables
    log1 = logger1.FindLog(context)
    log2 = logger2.FindLog(context)
    log3 = logger3.FindLog(context)
    
    error = np.abs(log2.data()[:na,:].transpose() - np.tile(qd,(log3.data().shape[1],1)))
    e_norm = np.array([np.linalg.norm(val) for val in error])
    for i in range(len(e_norm)):
        if np.all(e_norm[i:]<error_threshold):
            break
    final_error = e_norm[-1]
    # CYCLE TIME
    ts = log2.sample_times()[i]
    
    omega_j = []
    for i in range(log2.data().shape[1]):
        omega_j.append(log2.data()[:,i][na:])
    omega_j = np.array(omega_j).transpose()
    omega_m = omega_j*dv['gear_ratio']
    # Convert the motor speed to RPM
    omega_m_rpm = omega_m*60/(2*np.pi)
    # Motor torque realised
    tau_m = log3.data()/dv['gear_ratio']
    
    K_joule = 1/np.sqrt(dv['Km'])*np.identity(na)
    # Total power losses
    inst_friction_power = []
    inst_joule_power = []
    for i in range(len(log3.sample_times())):
        omega_inst = omega_m[:, i]
        tau_f = dv['tau_mu']*np.sign(omega_inst)+dv['b0']*omega_inst
        P_f = np.dot(tau_f, omega_inst)
        inst_friction_power.append(P_f)
        tau_t = tau_f+tau_m[:, i]
        P_t = np.dot(K_joule.dot(tau_t), tau_t)
        inst_joule_power.append(P_t)
    
    # POWER LOSSES
    total_friction_loss = np.sum(inst_friction_power)*time_step
    total_joule_loss = np.sum(inst_joule_power)*time_step
    
    results = np.array([workspace, 0, 0, 0, ts, total_friction_loss, total_joule_loss, final_error])

    # Angular angular velocity of the motors in RPM
    motor_rpm = omega_j*gear_ratio*60/(2*np.pi)
    plt.plot(log2.sample_times(), motor_rpm[0, :], log2.sample_times(), motor_rpm[1, :])
    plt.title('motor RPM vs time')

    # Motor torque demanded
    fig = plt.figure()
    plt.plot(log1.sample_times(), log1.data()[0, :], log1.sample_times(), log1.data()[1, :])
    plt.title("motor torque demanded vs time")

    # Motor torque demanded
    fig = plt.figure()
    plt.plot(log1.sample_times(), log3.data()[0, :], log1.sample_times(), log3.data()[1, :])
    plt.title("motor torque realised vs time")

    print('Max realised torque', np.max(np.abs(log3.data())))

    return results

./urdfs/divar3v7308465156.urdf
pre_val:  {'tau_mu': 0.0053, 'b0': 1.5e-05, 'k_velocity': 1.0606085407643906} dv_names:  ['omega_max_m', 'gear_ratio', 'Kp', 'Kd', 'm_m', 'a1', 'lua', 'a23_distr', 'lfa', 'a45_distr', 'a7']
Sample-DV:  [4750, 10, 15, 7.745966692414834, 0.452, 0.36, 0.36, 0.5, 0.36, 0.5, 0.1]
dv_bounds:  ([1000, 0.1, 0, 0, 0.001, 0.01, 0.1, 0.1, 0.1, 0.1, 0.1], [9000, 25, 100, 100, 2, 0.5, 0.5, 0.9, 0.5, 0.9, 0.25])
